 # Recommendation Approach
Since the goal is to build recommendations based on browsing and purchase history, we’ll focus on:

1. User-based Collaborative Filtering (recommending products similar users bought)
2. Item-based Collaborative Filtering (recommending products similar to what a user bought)
3. Hybrid Model (combining both approaches)


# User-Based Recommendation (Collaborative Filtering)

Concept:
* Observe User A's purchase history
* Find similar users (who purchased similar products)
* Identify products that similar users bought but User A has not purchased yet
* Recommend these products

Process:
1. Identify User A's purchase history (list of purchased products)
2. Find other users with similar purchase behavior
3. Extract products these similar users purchased that User A has not bought yet
4. Rank these products by popularity among similar users
5. Recommend the top-ranked products to User A

Best for recommending products based on what similar customers purchased
Works well when users have overlapping purchase patterns

# Item-Based Recommendation (Collaborative Filtering)

Concept:
* Look at the most recently purchased product by User A
* Identify how other users interact with this product
* Find other products that frequently appear together with this product in purchases
* Recommend those products to User A

Process:
1. Get the most recently purchased product by User A
2. Identify other users who purchased this product
3. Find what other products they also purchased
4. Identify the most commonly co-purchased products
5. Recommend the top-ranked products to User A

Best for recommending complementary or similar products
Works well when users frequently purchase related products

# Hybrid Recommendation (User + Item)

* Find similar users and recommend their most purchased products (User-Based)
* Find similar products based on purchase behavior (Item-Based)
* Combine both and rank based on purchase frequency & relevance

Balanced approach that improves recommendation diversity

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

# print files
folder_path = "/content/drive/MyDrive/Trafilea files"
print("Files inside:", os.listdir(folder_path))


Files inside: ['product_category_name_translation_cleaned.csv', 'customers_cleaned.csv', 'order_items_cleaned.csv', 'order_reviews_cleaned.csv', 'orders_cleaned.csv', 'products_cleaned.csv']


## Load csv files

In [4]:
import pandas as pd

# Load CSV files into Pandas DataFrames
orders = pd.read_csv(f"{folder_path}/orders_cleaned.csv")
order_items = pd.read_csv(f"{folder_path}/order_items_cleaned.csv")
customers = pd.read_csv(f"{folder_path}/customers_cleaned.csv")
products = pd.read_csv(f"{folder_path}/products_cleaned.csv")
product_category = pd.read_csv(f"{folder_path}/product_category_name_translation_cleaned.csv")
order_reviews = pd.read_csv(f"{folder_path}/order_reviews_cleaned.csv")

# Confirm successful loading
print("All CSV files loaded successfully!")


All CSV files loaded successfully!


## Preview data

In [5]:
# view dataframes
print("Orders Sample:")
display(orders.head())

print("Order Items Sample:")
display(order_items.head())

print("Customers Sample:")
display(customers.head())

print("Products Sample:")
display(products.head())

print("Order Reviews Sample:")
display(order_reviews.head())


Orders Sample:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


Order Items Sample:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


Customers Sample:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,Franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,Sao Bernardo Do Campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,Sao Paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,Mogi Das Cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,Campinas,SP


Products Sample:


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4,625.0,20.0,17.0,13.0


Order Reviews Sample:


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,no title,no message,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,no title,no message,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,no title,no message,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,no title,recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,no title,parabéns lojas lannister adorei comprar pela i...,2018-03-01 00:00:00,2018-03-02 10:26:53


## Merging the dataset

In [6]:
# Merge orders with customers to associate customer_id with unique users
orders_customers = orders.merge(customers, on="customer_id", how="inner")

# Merge order_items with orders_customers to associate products with customer_id
orders_products = order_items.merge(orders_customers, on="order_id", how="inner")

# Merge products with orders_products to add product attributes
orders_products_details = orders_products.merge(products, on="product_id", how="left")

# Merge with category translation for readable categories
orders_products_details = orders_products_details.merge(
    product_category, on="product_category_name", how="left"
)

# Drop unnecessary columns
orders_products_details.drop(columns=["product_category_name"], inplace=True)

# Rename column for clarity
orders_products_details.rename(columns={"product_category_name_english": "product_category"}, inplace=True)

# Merge with order reviews to associate reviews with purchases
orders_products_reviews = orders_products_details.merge(order_reviews, on="order_id", how="left")

# Display merged data sample
print("Merged Data Sample:")
display(orders_products_reviews.head())

# Check final shape
print("Final Merged DataFrame Shape:", orders_products_reviews.shape)


Merged Data Sample:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_length_cm,product_height_cm,product_width_cm,product_category,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,28.0,9.0,14.0,cool_stuff,97ca439bc427b48bc1cd7177abe71365,5.0,no title,"perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,50.0,30.0,40.0,pet_shop,7b07bacd811c4117b742569b04ce3580,4.0,no title,no message,2017-05-13 00:00:00,2017-05-15 11:34:13
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,33.0,13.0,33.0,furniture_decor,0c5b33dea94867d1ac402749e5438e8b,5.0,no title,chegou antes do prazo previsto e o produto sur...,2018-01-23 00:00:00,2018-01-23 16:06:31
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,16.0,10.0,15.0,perfumery,f4028d019cb58564807486a6aaf33817,4.0,no title,no message,2018-08-15 00:00:00,2018-08-15 16:39:01
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,35.0,40.0,30.0,garden_tools,940144190dcba6351888cafa43f3a3a5,5.0,no title,gostei pois veio no prazo determinado .,2017-03-02 00:00:00,2017-03-03 10:54:59


Final Merged DataFrame Shape: (113314, 32)


In [7]:
# Save merged dataset for easy import
orders_products_reviews.to_csv("orders_products_reviews_cleaned.csv", index=False)

print("Merged dataset saved as 'orders_products_reviews_cleaned.csv'")


Merged dataset saved as 'orders_products_reviews_cleaned.csv'


In [8]:
# Load the cleaned dataset
df = pd.read_csv("orders_products_reviews_cleaned.csv")

# Check the first few rows
print("Merged Dataset Sample:")
display(df.head())

# Check shape
print("Dataset Shape:", df.shape)


Merged Dataset Sample:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,product_length_cm,product_height_cm,product_width_cm,product_category,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,28.0,9.0,14.0,cool_stuff,97ca439bc427b48bc1cd7177abe71365,5.0,no title,"perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,50.0,30.0,40.0,pet_shop,7b07bacd811c4117b742569b04ce3580,4.0,no title,no message,2017-05-13 00:00:00,2017-05-15 11:34:13
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,33.0,13.0,33.0,furniture_decor,0c5b33dea94867d1ac402749e5438e8b,5.0,no title,chegou antes do prazo previsto e o produto sur...,2018-01-23 00:00:00,2018-01-23 16:06:31
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,16.0,10.0,15.0,perfumery,f4028d019cb58564807486a6aaf33817,4.0,no title,no message,2018-08-15 00:00:00,2018-08-15 16:39:01
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,35.0,40.0,30.0,garden_tools,940144190dcba6351888cafa43f3a3a5,5.0,no title,gostei pois veio no prazo determinado .,2017-03-02 00:00:00,2017-03-03 10:54:59


Dataset Shape: (113314, 32)


In [9]:
# Count the number of ratings per product
product_rating_counts = df.groupby("product_id")["review_score"].count().reset_index()

# Rename column for clarity
product_rating_counts.rename(columns={"review_score": "rating_count"}, inplace=True)

# Display descriptive statistics
print("Review Count Summary:\n", product_rating_counts["rating_count"].describe())



Review Count Summary:
 count    32951.000000
mean         3.410276
std         10.605462
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max        524.000000
Name: rating_count, dtype: float64


In [10]:
# Show the top 20 most-rated products
print("\nTop 20 Most Rated Products:")
print(product_rating_counts.sort_values(by="rating_count", ascending=False).head(20))



Top 20 Most Rated Products:
                             product_id  rating_count
22112  aca2eb7d00ea1a7b8ebd4e68314663af           524
8613   422879e10f46682990de24d770e7f83d           486
19742  99a4788cb24856965c36a24e339b6058           482
7364   389d119b48cf3043d311335e499d9c6b           391
7079   368c6c730842d78016ad823897a372db           388
10840  53759a2ecddad2bb87a079a1f1519f73           373
27039  d1c427060a0f73f6b889a5c7c61f2ac4           340
10867  53b36df67ebb7c41585e8d54d6772e08           320
2794   154e7e31ebfa092203795c972e5804a6           292
8051   3dd2a17168ec895c781a9191c1e95ad7           272
5692   2b4609f8948be18874494203496bc318           269
16086  7c1bd920dbdf22470b68bde975dd3ccf           235
21302  a62e25e09e05e6faf31d90c6ec1aa3d1           225
24086  bb50f2e236e5eea0100680137654686c           196
11746  5a848e4ab52fd5445cdc07aab1c40e48           195
28970  e0d64dcfaa3b6db5c54ca298ae101d05           193
29513  e53e557d5a159f5aa2c5e995dfdf244b           185

In [11]:
# Check the distribution of average review scores
import pandas as pd
from IPython.display import display

average_review_scores = df.groupby("product_id")["review_score"].mean().reset_index()

# Count how many products have an average rating below 3.0
low_rated_products_count = (average_review_scores["review_score"] < 3.0).sum()

# Get percentage of low-rated products
total_products = len(average_review_scores)
low_rated_percentage = (low_rated_products_count / total_products) * 100

# Display summary
low_rated_summary = pd.DataFrame({
    "Total Products": [total_products],
    "Products with Review Score < 3.0": [low_rated_products_count],
    "Percentage of Low-Rated Products": [low_rated_percentage]
})

# Display results
print("Low-Rated Products Summary")
display(low_rated_summary)

Low-Rated Products Summary


,Total Products,Products with Review Score < 3.0,Percentage of Low-Rated Products
0,32951,4421,13.416892


## Filter Out Products with review_score < 3.0

In [12]:
# Remove low-rated products
df_filtered = df[df["product_id"].isin(average_review_scores[average_review_scores["review_score"] >= 3.0]["product_id"])]

# Check the shape after filtering
print("Dataset Shape After Removing Low-Rated Products:", df_filtered.shape)

# Save the new filtered dataset
df_filtered.to_csv("filtered_orders_products_reviews_above_3.csv", index=False)
print("Filtered dataset saved as 'filtered_orders_products_reviews_above_3.csv'")


Dataset Shape After Removing Low-Rated Products: (103755, 32)
Filtered dataset saved as 'filtered_orders_products_reviews_above_3.csv'


## Select only frequently bought products as products with less frequency may not have enough interaction data for collaborative filtering

In [13]:
# Count the number of purchases per product
product_purchase_counts = df_filtered["product_id"].value_counts()

In [14]:
# get summary
product_purchase_counts.describe()

,count
count,28368.000000
mean,3.657466
std,11.425392
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,527.000000


## determine a reasonable cutoff, by finding the 90th percentile of purchases

In [15]:
# Get the 90th percentile of purchase frequency
purchase_threshold = product_purchase_counts.quantile(0.90)
print(f"90th Percentile Purchase Threshold: {purchase_threshold}")


90th Percentile Purchase Threshold: 7.0


Our goal is to build a high-quality recommendation system based on purchase history and browsing behavior (though i don’t have explicit browsing data, purchases act as implicit feedback).

Since 90% of products were purchased fewer than 7 times, i am keeping only the top 10%.
This ensures high-quality recommendations, but may limit diversity.
i will lower the threshold to 5 for diversity

In [16]:
# Keep only products purchased at least 5 times
df_filtered = df_filtered[df_filtered["product_id"].isin(product_purchase_counts[product_purchase_counts >= 5].index)]

# Check dataset shape after filtering
print("Dataset Shape After Removing Low-Purchase Products:", df_filtered.shape)

# Save the dataset
df_filtered.to_csv("filtered_orders_products_reviews_above_3_purchased_5.csv", index=False)
print("Filtered dataset saved as 'filtered_orders_products_reviews_above_3_purchased_5.csv'")


Dataset Shape After Removing Low-Purchase Products: (65981, 32)
Filtered dataset saved as 'filtered_orders_products_reviews_above_3_purchased_5.csv'


## Observation

- Original Dataset Shape: (113,314, 32)
- After Removing Low-Rated Products (review_score < 3.0): (103,755, 32)
- After Removing Low-Purchase Products (purchases < 5): (65,981, 32)
- Total Reduction: ~41.8% of the data removed, keeping only relevant,high-quality products.


* Keeps only well-reviewed products (review_score ≥ 3.0)
* Removes rarely purchased products (purchases ≥ 5)
* Retains enough products for diverse recommendations
* Balances recommendation accuracy and variety

In [17]:
# Count the number of purchases per customer
customer_purchase_counts = df_filtered["customer_id"].value_counts()

# Keep only customers who made at least 2 purchases
df_filtered = df_filtered[df_filtered["customer_id"].isin(customer_purchase_counts[customer_purchase_counts >= 2].index)]

# Check dataset shape after filtering
print("Dataset Shape After Removing Inactive Customers:", df_filtered.shape)

# Save the dataset
df_filtered.to_csv("filtered_orders_products_reviews_active_customers.csv", index=False)
print("Filtered dataset saved as 'filtered_orders_products_reviews_active_customers.csv'")


Dataset Shape After Removing Inactive Customers: (13838, 32)
Filtered dataset saved as 'filtered_orders_products_reviews_active_customers.csv'


In [18]:
# Make a copy of the already filtered dataset for easy reference
df1 = df_filtered.copy()

# Save it as a backup
df1.to_csv("filtered_orders_products_reviews_active_customers_backup.csv", index=False)

print("Backup dataset saved as 'filtered_orders_products_reviews_active_customers_backup.csv'")


Backup dataset saved as 'filtered_orders_products_reviews_active_customers_backup.csv'


In [19]:
# view the data columns
df1.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'product_category',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

## Create a pivot table based on purchase history

In [20]:

# Ensure interaction matrix includes only high-rated products
high_rated_products = df_filtered[df_filtered["review_score"] >= 3.0]["product_id"].unique()
df_filtered_high_rated = df_filtered[df_filtered["product_id"].isin(high_rated_products)]

# Create a new interaction matrix excluding low-rated products
interaction_matrix = df_filtered_high_rated.pivot_table(
    index="customer_id",
    columns="product_id",
    values="order_id",
    aggfunc="count"
).fillna(0)

# Display new interaction matrix shape
print("Updated Interaction Matrix Shape (Excluding Low-Rated Products):", interaction_matrix.shape)

# Show a sample of the matrix
interaction_matrix.head()


Updated Interaction Matrix Shape (Excluding Low-Rated Products): (5472, 2281)


product_id,001795ec6f1b187d37335e1c4704762e,001b72dfd63e9833e8c02742adf472e3,00210e41887c2a8ef9f791ebc780cc36,00250175f79f584c14ab5cecd80553cd,002af88741ba70c7b5cf4e4a0ad7ef85,005030ef108f58b46b78116f754d8d38,008cff0e5792219fae03e570f980b330,00ab8a8b9fe219511dc3f178c6d79698,00ba6d766f0b1d7b78a5ce3e1e033263,00faa46f36261af8bbf3a4d37fa4841b,...,ff29d8cb1cd0cd5ea37b80dac9939e1c,ff2c1ec09b1bb340e84f0d6b21cc7dbb,ff55834c0b3272853dfe62340f2bb91b,ff5d7f21d255435967dfbe72b5dcdb57,ff5f8606556b625613f069a5a9a48966,ffa7e0cbe11656d11a117b534bb1db27,ffaf0af7eebb57c7f262b51ebb05dfd6,ffc9caf33e2d1e9f44e3e06da19085f7,ffef256879dbadcab7e77950f4f4a195,fff0a542c3c62682f23305214eaeaa24
customer_id,,,,,,,,,,,,,,,,,,,,,
00066ccbe787a588c52bd5ff404590e3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0033823ee55671ac5317d423291333c2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00426311a53f3c052943c88b692a3be2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005596395ca3acd0433fe7d6f95a028e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0058ebe2dc136d918dd001968cfa5903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## check sparsity

In [21]:
import numpy as np

# Calculate total number of cells in the matrix
total_cells = np.product(interaction_matrix.shape)

# Count the number of nonzero values (actual purchases)
nonzero_cells = np.count_nonzero(interaction_matrix)

# Compute sparsity
sparsity = 1 - (nonzero_cells / total_cells)

print(f"Dataset Sparsity: {sparsity:.4f}")


Dataset Sparsity: 0.9995


The data is very sparsed, Since our goal is to make recommendations based on browsing & purchase history, the best approach is:

✔ Step 1: Remove rare products (purchased by <3 users) to reduce sparsity.

✔ Step 2: Show best-sellers to cold-start users to prevent empty recommendations.

✔ Step 3: Use a hybrid model (collaborative + content-based filtering) to improve accuracy.



In [22]:
# Count how many unique customers bought each product
product_buyer_counts = (interaction_matrix > 0).sum(axis=0)

# Set threshold (keep products purchased by at least 3 users)
product_threshold = 3
popular_products = product_buyer_counts[product_buyer_counts >= product_threshold].index

# Keep only popular products in the interaction matrix
interaction_matrix = interaction_matrix[popular_products]

# Check new matrix shape after removing rare products
print("Updated Interaction Matrix Shape:", interaction_matrix.shape)


Updated Interaction Matrix Shape: (5472, 665)


## Fill Missing Values with Best-Sellers for Cold-Start Users
What this does:

- Extracts top 10 most purchased products from df1.
- These products will be recommended to new users with no purchase history.

In [23]:
# Get top 10 best-selling products
top_products = df1["product_id"].value_counts().head(10).index.tolist()

# Display best-selling products
print("Top 10 Best-Selling Products:", top_products)


Top 10 Best-Selling Products: ['422879e10f46682990de24d770e7f83d', '368c6c730842d78016ad823897a372db', 'aca2eb7d00ea1a7b8ebd4e68314663af', '53759a2ecddad2bb87a079a1f1519f73', '389d119b48cf3043d311335e499d9c6b', 'b532349fe46b38fbc7bb3914c1bdae07', 'e53e557d5a159f5aa2c5e995dfdf244b', '9ecadb84c81da840dbf3564378b586e9', 'eb8c629f70275fd1c4f809116cce1efc', '36f60d45225e60c7da4558b070ce4b60']


## Step 1: Compute User-User Similarity
Now, we compute the similarity between users based on their purchase history.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Compute User-User Similarity Matrix
user_similarity = cosine_similarity(interaction_matrix)

# Convert to DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

# Display a sample
print(user_similarity_df.head())


customer_id                       00066ccbe787a588c52bd5ff404590e3  \
customer_id                                                          
00066ccbe787a588c52bd5ff404590e3                               1.0   
0033823ee55671ac5317d423291333c2                               0.0   
00426311a53f3c052943c88b692a3be2                               0.0   
005596395ca3acd0433fe7d6f95a028e                               0.0   
0058ebe2dc136d918dd001968cfa5903                               0.0   

customer_id                       0033823ee55671ac5317d423291333c2  \
customer_id                                                          
00066ccbe787a588c52bd5ff404590e3                               0.0   
0033823ee55671ac5317d423291333c2                               0.0   
00426311a53f3c052943c88b692a3be2                               0.0   
005596395ca3acd0433fe7d6f95a028e                               0.0   
0058ebe2dc136d918dd001968cfa5903                               0.0   

customer_id       

## Function to Recommend Products Based on Similar Users
we dont have so much similarity so lets check if we can still recommend products purchased by similar users.

In [25]:
# Function to recommend products for a given user with additional filtering
def recommend_products_for_user(user_id, top_n=5):
    if user_id not in user_similarity_df.index:
        return f"New user! Recommend best-sellers: {top_products}"

    # Get similar users (excluding the user themselves)
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:6].index

    # Get products purchased by similar users
    recommended_product_ids = interaction_matrix.loc[similar_users].sum().sort_values(ascending=False).index[:top_n]

    # Merge with product details
    recommended_products = df1[df1["product_id"].isin(recommended_product_ids)][["product_id", "product_category", "price", "review_score"]].drop_duplicates()

    # Apply final filtering to remove low-rated products
    recommended_products = recommended_products[recommended_products["review_score"] >= 3.0]

    return recommended_products

# Test recommendation after filtering
user_id = interaction_matrix.index[0]
print(f"Recommended Products for {user_id}:")
display(recommend_products_for_user(user_id))


Recommended Products for 00066ccbe787a588c52bd5ff404590e3:


,product_id,product_category,price,review_score
13,368c6c730842d78016ad823897a372db,garden_tools,49.90,5.0
1858,368c6c730842d78016ad823897a372db,garden_tools,53.90,5.0
3528,ab1f9387c0627dd24000bfbc54fdeee0,electronics,12.25,5.0
4175,368c6c730842d78016ad823897a372db,garden_tools,49.00,3.0
4926,368c6c730842d78016ad823897a372db,garden_tools,59.90,5.0
5466,368c6c730842d78016ad823897a372db,garden_tools,59.90,4.0
16062,368c6c730842d78016ad823897a372db,garden_tools,59.00,5.0
20502,ab1106e227a745836a0dc83b374e2575,food,19.99,4.0
22422,368c6c730842d78016ad823897a372db,garden_tools,49.00,5.0
25828,368c6c730842d78016ad823897a372db,garden_tools,49.90,3.0


## Step 1: Select a Random User
Pick a random user for testing while ensuring reproducibility.

In [26]:
import random

# Set a fixed random state for consistency
random.seed(42)

# Select a random user
user_id = random.choice(interaction_matrix.index.tolist())

print(f"Selected User for Recommendation: {user_id}")


Selected User for Recommendation: f58fc8280d4679f90ba28d39b64c61ca


## Step 2: Look at the User’s Purchase Preferences
Retrieve the user’s purchase history.

In [27]:
# Retrieve user purchase history
user_purchase_history = df1[df1["customer_id"] == user_id][["product_id", "product_category"]].drop_duplicates()

print(f"Purchase History for User {user_id}:")
display(user_purchase_history)


Purchase History for User f58fc8280d4679f90ba28d39b64c61ca:


,product_id,product_category
103891,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business


# Step 3: Select Other Users with Similar Purchase History
Find top 5 similar users using the User-User Similarity Matrix.

In [28]:
# Get similar users (excluding the user themselves)
similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:6].index

print(f"Top 5 Similar Users to {user_id}:")
print(similar_users)


Top 5 Similar Users to f58fc8280d4679f90ba28d39b64c61ca:
Index(['4a1bdf4fac99c14d55e6a7caf7068d46', '7302aa13024a3e490ad2f22b23bb2bc1',
       'a95d66809480de437e5e93ad6cda31cf', 'a9b440dd10180f222e61cb9016eec6c4',
       'a9929fa5f092ace247a1ea0a3469bbee'],
      dtype='object', name='customer_id')


## Step 4: Rank Products Purchased by Similar Users
Check what similar users bought and rank products.

In [29]:
# Get all products purchased by similar users
similar_users_purchases = df1[df1["customer_id"].isin(similar_users)][["product_id", "product_category", "price", "review_score"]]

# Rank products by popularity among similar users
ranked_products = similar_users_purchases.groupby(["product_id", "product_category"]).size().reset_index(name="purchase_count")

# Sort by most purchased items
ranked_products = ranked_products.sort_values("purchase_count", ascending=False)

print("Top Ranked Products from Similar Users:")
display(ranked_products.head(5))


Top Ranked Products from Similar Users:


,product_id,product_category,purchase_count
2,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business,3
1,7402c8cf69df715729b26da0ac2cc721,small_appliances,2
0,1de1332e38dc2f575edfeae165edea4e,bed_bath_table,1
3,aa70ae9ee9a798d44ae6e00ce6d82cb1,health_beauty,1
4,ba131cee7989ef04b93a79030969c92f,furniture_decor,1


# Step 5: Filter and Recommend the Best Products
Filter out low-rated products and avoid duplicates.



In [30]:
# Step 1: Retrieve user's past purchase categories
user_categories = user_purchase_history["product_category"].unique()

# Step 2: Filter recommended products based on user's purchase history categories
recommended_products = ranked_products[ranked_products["product_category"].isin(user_categories)]

# Step 3: Merge product details (price, review_score) & remove duplicates
recommended_products = recommended_products.merge(
    df1[["product_id", "price", "review_score"]],
    on="product_id", how="left"
).drop_duplicates()

# Step 4: Remove low-rated products (below 3.0 review score)
recommended_products = recommended_products[recommended_products["review_score"] >= 3.0]

# Step 5: Keep only the best-priced version of each product
recommended_products = recommended_products.sort_values(["review_score", "price"], ascending=[False, True])

# Step 6: Drop duplicates based on product_id & price (prevents duplicate products with the same price)
recommended_products = recommended_products.drop_duplicates(subset=["product_id", "price"])

# Step 7: If fewer than 5 recommendations exist, expand to high-rated products (≥3.0) from any category
if len(recommended_products) < 5:
    extra_products = df1[
        (df1["review_score"] >= 3.0) &
        (~df1["product_category"].isin(user_categories))  # Ensure these are from new categories
    ]

    # Prevent adding duplicates of the same product at the same price
    recommended_products = pd.concat([recommended_products, extra_products]).drop_duplicates(subset=["product_id", "price"])

# Step 8: Limit to top N recommendations
top_n = min(5, len(recommended_products))
display_columns = ["product_id", "product_category", "price", "review_score"]
final_recommendations = recommended_products[display_columns].head(top_n)

# Step 9: Print & display final recommendations
print(f"Final Recommendations for User {user_id}:")
display(final_recommendations)


Final Recommendations for User f58fc8280d4679f90ba28d39b64c61ca:


,product_id,product_category,price,review_score
7,d04bbac48960ecb7ea311b00ca6e1cb7,industry_commerce_and_business,98.0,5.0
6,d04bbac48960ecb7ea311b00ca6e1cb7,industry_commerce_and_business,104.0,5.0
3,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business,110.0,5.0
4,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business,99.0,4.0
13,368c6c730842d78016ad823897a372db,garden_tools,49.9,5.0


In [31]:
# Step 10: Save the single-user recommendations for later use
final_recommendations.to_csv(f"single_user_recommendation_{user_id}.csv", index=False)

In [32]:
# Store User-Based Recommendations
user_based_recommendations_df = final_recommendations.copy()

## Item based recommendation

## Create the Product-Product Similarity Matrix

use consine similarity because our data is sparse

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Compute product-product similarity using Cosine Similarity
product_similarity_matrix = pd.DataFrame(
    cosine_similarity(interaction_matrix.T),
    index=interaction_matrix.columns,
    columns=interaction_matrix.columns
)

# Check the shape of the similarity matrix
print("Product-Product Similarity Matrix Shape:", product_similarity_matrix.shape)

product_similarity_matrix.head()

Product-Product Similarity Matrix Shape: (665, 665)


product_id,001b72dfd63e9833e8c02742adf472e3,005030ef108f58b46b78116f754d8d38,008cff0e5792219fae03e570f980b330,00ba6d766f0b1d7b78a5ce3e1e033263,013e6676e0e3529e5909ff54370daddf,013ee64977aaa6b2b25475095162e0e9,014a8a503291921f7b004a5215bb3c36,0152f69b6cf919bcdaf117aa8c43e5a2,017692475c1c954ff597feda05131d73,01d4157fd9738ea81038316ae4cfc192,...,fbce4c4cb307679d89a3bf3d3bb353b9,fbd4031ddbc3d02b95834b62769d3ad0,fc1449f3ffc59fd2eb9640505aea0581,fc1d8637c0268af3db482c14b7ef8e75,fc5dd987f12a7b823a76a44aa1ba88f6,fcaab5d7f656094e49fbe4ee3a506658,fd424396761d9e1609c5516b7d8ce17b,fe6a9515d655fa7936b8a7c841039f34,ff2c1ec09b1bb340e84f0d6b21cc7dbb,ffa7e0cbe11656d11a117b534bb1db27
product_id,,,,,,,,,,,,,,,,,,,,,
001b72dfd63e9833e8c02742adf472e3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005030ef108f58b46b78116f754d8d38,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
008cff0e5792219fae03e570f980b330,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ba6d766f0b1d7b78a5ce3e1e033263,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
013e6676e0e3529e5909ff54370daddf,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Function to Recommend Similar Products

In [34]:
def recommend_similar_products(product_id, top_n=5):
    """
    Given a product_id, recommend the top N most similar products.
    Prioritizes same-category products and ensures high quality.
    """
    if product_id not in product_similarity_matrix.index:
        return f"Product ID {product_id} not found in similarity matrix."

    # Retrieve the category of the input product
    product_category = df1[df1["product_id"] == product_id]["product_category"].values[0]

    # Retrieve similar products sorted by similarity score (descending)
    similar_products = product_similarity_matrix[product_id].sort_values(ascending=False)[1:].index

    # Fetch details of the recommended products
    recommended_products = df1[df1["product_id"].isin(similar_products)][["product_id", "product_category", "price", "review_score"]].drop_duplicates()

    # Prioritize same-category products first
    same_category_products = recommended_products[recommended_products["product_category"] == product_category]

    # If fewer than top_n exist, include high-rated (≥3.0) products from any category
    if len(same_category_products) < top_n:
        extra_products = recommended_products[recommended_products["review_score"] >= 3.0]
        recommended_products = pd.concat([same_category_products, extra_products]).drop_duplicates()
    else:
        recommended_products = same_category_products

    # Keep only the best-priced version of each product
    recommended_products = recommended_products.sort_values(["review_score", "price"], ascending=[False, True]).drop_duplicates(subset="product_id")

    # Limit to top N recommendations
    final_recommendations = recommended_products.head(top_n)

    return final_recommendations


In [35]:
# Choose a product ID to test
test_product_id = df1["product_id"].sample(1).values[0]

# Get similar product recommendations
print(f"Recommended Products for Product ID {test_product_id}:")
display(recommend_similar_products(test_product_id))


Recommended Products for Product ID 0152f69b6cf919bcdaf117aa8c43e5a2:


,product_id,product_category,price,review_score
79824,d7205c0ebebe2744d7c2e44b6d69cc95,bed_bath_table,8.99,5.0
53464,4e1346d7b7e02c737a366b086462e33e,bed_bath_table,11.99,5.0
20870,642a40d4d73b4b2604968937de13634a,bed_bath_table,12.90,5.0
51979,730f1927f42fda4370209c28203eb0ab,bed_bath_table,13.99,5.0
43611,7736ea239e147291f6adb766c992beb1,bed_bath_table,15.90,5.0


## Save Item-Based Recommendations

In [36]:
# Save the item-based similarity matrix
product_similarity_matrix.to_csv("product_similarity_matrix.csv")
print("\n Product similarity matrix saved as 'product_similarity_matrix.csv'")



 Product similarity matrix saved as 'product_similarity_matrix.csv'


In [37]:
# Save Item-Based Similarity Matrix
item_based_similarity_df = product_similarity_matrix.copy()

In [38]:
test_product_category = df1[df1["product_id"] == test_product_id]["product_category"].values[0]
print(f"Test Product Category: {test_product_category}")


Test Product Category: bed_bath_table


In [39]:
# Store Item-Based Recommendations (Top similar products)
item_based_recommendations_df = recommend_similar_products(test_product_id)


# Hybrid Recommendation System
I am implementing the weighted hybrid approach, ensuring:

* User-based recommendations dominate (70% weight).
* Item-based recommendations fill gaps (30% weight).
* Duplicates are removed, and sorting is optimized.

because of my use case, Considering the challenges we faced with the data:

-  Data sparsity → User-based filtering is limited for users with few purchases.

- Cold-start problem → Some users and products have low interaction history.
- Product diversity → Some products are dominated by a single category.
- Duplicate product recommendations → We had issues with multiple sellers offering the same product.
-  Price variations → Users need to see different price points rather than just filtering the lowest.



In [40]:
# Ensure user-based and item-based recommendations are DataFrames
user_based_recommendations_df = pd.DataFrame(user_based_recommendations_df,
                                             columns=["product_id", "product_category", "price", "review_score"])
item_based_recommendations_df = pd.DataFrame(item_based_recommendations_df,
                                             columns=["product_id", "product_category", "price", "review_score"])

# Save user-based and item-based recommendations
user_based_recommendations_df.to_csv(f"user_based_recommendations_{user_id}.csv", index=False)
item_based_recommendations_df.to_csv(f"item_based_recommendations_{test_product_id}.csv", index=False)


In [41]:
# Step 1: Merge User-Based and Item-Based Recommendations
hybrid_recommendations = pd.concat([user_based_recommendations_df, item_based_recommendations_df], ignore_index=True).drop_duplicates(subset=["product_id"])

# Step 2: Retrieve User’s Purchase Category
user_categories = user_purchase_history["product_category"].unique()

# Step 3: Prioritize Products from User’s Purchase Categories
category_filtered = hybrid_recommendations[hybrid_recommendations["product_category"].isin(user_categories)]

# Step 4: If Not Enough, Add More Products from the Same Category
if len(category_filtered) < 5:
    extra_products = df1[
        (df1["product_category"].isin(user_categories)) &  # Ensure products are from user's category
        (df1["review_score"] >= 3.0)  # Maintain high-quality recommendations
    ].drop_duplicates(subset=["product_id"]).head(5 - len(category_filtered))

    category_filtered = pd.concat([category_filtered, extra_products]).drop_duplicates(subset=["product_id"])

# Step 5: Sort by Review Score & Price
category_filtered = category_filtered.sort_values(["review_score", "price"], ascending=[False, True])

# Step 6: Limit to Top 5 Recommendations
final_hybrid_recommendations = category_filtered.head(5)

# Step 7: Choose Columns to Display
display_columns = ["product_id", "product_category", "price", "review_score"]
final_hybrid_recommendations = final_hybrid_recommendations[display_columns]  # Selecting specific columns

# Step 8: Save Recommendations
hybrid_recommendations_file = f"hybrid_recommendations_{user_id}.csv"
final_hybrid_recommendations.to_csv(hybrid_recommendations_file, index=False)

# Step 9: Display Results
print(f"Hybrid recommendations saved as '{hybrid_recommendations_file}'")
display(final_hybrid_recommendations)


Hybrid recommendations saved as 'hybrid_recommendations_f58fc8280d4679f90ba28d39b64c61ca.csv'


,product_id,product_category,price,review_score
3168,9b4d597238711e4dfc04d98bb2e5ed17,industry_commerce_and_business,76.0,5.0
0,d04bbac48960ecb7ea311b00ca6e1cb7,industry_commerce_and_business,98.0,5.0
2,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business,110.0,5.0
6612,8246b1e9198f46fc3c05900f43568ba6,industry_commerce_and_business,38.0,4.0
3760,593236d0ff46b4299b4787fb8d43f7f0,industry_commerce_and_business,69.0,3.0


In [42]:
# Retrieve the user's purchase history
user_purchase_history = df1[df1["customer_id"] == user_id][["product_id", "product_category", "price", "review_score"]].drop_duplicates()

# Display the user's past purchases
print(f"Purchase History for User {user_id}:")
display(user_purchase_history)


Purchase History for User f58fc8280d4679f90ba28d39b64c61ca:


,product_id,product_category,price,review_score
103891,88e84a987b4681434dedc0bc9e00826b,industry_commerce_and_business,110.0,1.0
